# **[GD-16] HuggingFace 커스텀 프로젝트**

# **1. GLUE dataset과 Huggingface**
---

## **1.1 GLUE Benchmark Dataset**

pretrained model의 성능을 측정하기 위해 classification, summarization, reasoning, Q&A 등 NLP 모델의 성능을 평가할 수 있는 다양한 task를 해당 모델 하나만을 이용해 모두 수행해 보면서 종합적인 성능을 논하는 것이 일반화되었다.

그 중 NLP 모델의 성능을 측정하기 위한 데이터셋으로 `GLUE benchmark Dataset`이 최근 활용되고 있다. 총 10가지 데이터셋이 있다.  
[General Language Understanding Evaluation(GLUE) benchmark Dataset](https://gluebenchmark.com/)  
- CoLA : 문법에 맞는 문장인지 판단  
- MNLI : 두 문장의 관계 판단(entailment, contradiction, neutral)  
- MNLI-MM : 두 문장이 안 맞는지 판단  
- MRPC : 두 문장의 유사도 평가  
- SST-2 : 감정분석  
- STS-B : 두 문장의 유사도 평가  
- QQP : 두 질문의 유사도 평가  
- QNLI : 질문과 paragraph 내 한 문장이 함의 관계(entailment)인지 판단  
- RTE : 두 문장의 관계 판단(entailment, not_entailment)  
- WNLI : 원문장과 대명사로 치환한 문장 사이의 함의 관계 판단  

## **1.2 Huggingface가 제공하는 GLUE task 예제 코드**

Huggingface와 같은 NLP framework는 해당 framework를 활용하여 새로 만들어진 모델의 성능을 빠르게 평가해 볼 수 있도록 하는 예제 코드를 제공하고 있다.

In [1]:
# $ cd ~/aiffel && git clone https://github.com/huggingface/transformers.git
# $ cd transformers && pip install -e .
# $ pip install datasets
# $ pip install huggingface-hub==0.1.0

In [2]:
# $ cd ~/aiffel/transformers
# $ python examples/tensorflow/text-classification/run_glue.py \
# 	--model_name_or_path bert-base-cased \
# 	--task_name mrpc \
# 	--output_dir ./models/mrpc \
# 	--overwrite_output_dir \
# 	--do_train \
# 	--do_eval \
# 	--num_train_epochs 1 \
# 	--save_steps 20000

위 코드는 10가지 GLUE task 중 `mrpc task`를 수행하는 예제코드이다.  
이 코드는 Huggingface의 framework 기반으로 BERT `bert-base-cased`을 활용하여 `mrpc task`를 수행한다.  
만약 `task_name` 및 다른 파라미터를 적절히 변경한 후 수행하면 GLUE task도 간단히 수행해 볼 수 있다.  
model도 다양하게 바꾸어보면서 수행 가능할 것이다.  
이 예제만으로도 NLP framework의 강력함을 느껴볼 수 있다.

GLUE 'mrpc' task를 나만의 커스텀 프로젝트로 구성해서 다시 해결해보자.  
이 과정을 통해 Huggingface framework에 대해 좀 더 명확하게 이해할 수 있을 것이다.

# **2. 커스텀 프로젝트 제작**
---

## **2.1 Processor**

### **2.1.1 mrpc 데이터셋 분석**
본격적으로 Huggingface framework를 활용해 보자.  
프로젝트를 수행하기 위한 첫 단계는 데이터를 분석하는 것이다.

In [3]:
!pip install transformers

In [4]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

print("슝~")

슝~


GLUE 데이터셋은 홈페이지에서 원본을 다운로드할 수도 있지만, 이번에는 `tensorflow_datasets`에서 제공하는 것을 이용해보자.

In [5]:
data, info = tfds.load('glue/mrpc', with_info=True)
info.splits['train'].num_examples

3668

- 3668개의 훈련 데이터셋이 존재한 것을 확인할 수 있다.

`data`는 `tf.data.Dataset`을 상속받은 클래스의 형태일 것이다.  
우선 1개의 데이터만가져다가 어떻게 생겼는지 확인해보자.

In [6]:
data['train'].take(1)

<TakeDataset element_spec={'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sentence1': TensorSpec(shape=(), dtype=tf.string, name=None), 'sentence2': TensorSpec(shape=(), dtype=tf.string, name=None)}>

실제 내용도 확인해보자.

In [7]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'The identical rovers will act as robotic geologists , searching for evidence of past water .', shape=(), dtype=string)
tf.Tensor(b'The rovers act as robotic geologists , moving on six wheels .', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


### **2.1.2 Processor의 활용**
이전 시간에 `Huggingface transformers`에서 task 별로 데이터셋을 가공하는 일반적인 클래스 구조인 `Processor`에 대해 다룬 바 있다.

아래는 추상 클래스인 `Processor`를 한번 상속받은 후, Sequence Classification task를 수행하는 모델의 `Processor` 추상 클래스인 `DataProcessor`이다.

In [8]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

print("슝~")

슝~


- 아직은 추상 클래스 상태이기 떄문에 그대로 사용하면 `NotImplementedError`를 발생시키는 메소드들이 포함되어 있다.  
- 이 메소드들을 오버라이드해야 실제 사용 가능한 클래스가 얻어진다.

아래는 'mrpc' 원본 데이터셋을 처리하여 모델에 입력할 수 있도록 정리해 주는 `MrpcProcessor` 클래스이다.

In [9]:
class MrpcProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

print("슝~")

슝~


`get_example_from_tensor_dict()` 메소드가 어떤 역할을 하게 되는지 살펴보자.

In [10]:
processor = MrpcProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'idx': <tf.Tensor: shape=(), dtype=int32, numpy=1680>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sentence1': <tf.Tensor: shape=(), dtype=string, numpy=b'The identical rovers will act as robotic geologists , searching for evidence of past water .'>, 'sentence2': <tf.Tensor: shape=(), dtype=string, numpy=b'The rovers act as robotic geologists , moving on six wheels .'>}
------processor 가공데이터------
InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')


- Processor는 'Raw Dataset를 Annotated Dataset으로 변환'하는 역할을 한다.
- 항목별로 `text_a`, `text_b`, `label` 등의 annotation이 포함된 `InputExample`로 변환되어 있음을 알 수 있다.

다음 코드는 `tfds_map()` 메소드를 활용한 경우이다.

In [11]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')


- 이전과 별 다른 차이는 없다.
- `tfds_map`는 label을 가공하는 메소드인데, 이미 숫자로 잘 가공된 label이기 때문에 특별한 변화가 없다.

실제 label을 확인하여 Binary Classification 문제로 잘 정의되고 있는지 확인해 보자.

In [12]:
label_list = processor.get_labels()
label_list

['0', '1']

In [13]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'0': 0, '1': 1}

## **2.2 Tokenizer와 Model**

Processor를 통해 Framework를 활용하여 데이터셋을 가공하는 작업을 잘 진행했다면 이미 절반 이상 진행한 것이다.  
NLP 모델링의 핵심을 이루는 Tokenizer와 Model은 framework에서 이미 잘 만들어져있는 것을 쉽게 가져다 쓸 수 있기 때문이다.

이전 스텝에서 만든 `MRPCProcessor` 클래스와 framework를 결합시켜 나가는 과정을 진행해보자.  
우선 아래와 같이 `tokenizer`와 `model`을 간단히 생성한다.

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


이제 processor와 tokenizer, 원본 데이터셋을 결합하여 model에 입력할 데이터셋을 생성해보자.

In [15]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

print("슝~")

슝~


In [16]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

print("슝~")

슝~


`_glue_convert_examples_to_features()` 함수는 processor가 생성한 example을 `tokenizer`로 인코딩하여 feature로 변환하는 역할을 한다.  
이후 `tf_glue_convert_examples_to_features()` 함수는 내부적으로 `_glue_convert_examples_to_features()`를 호출해서 얻은 feature를 바탕으로 `tf.data.Dataset`을 생성하여 리턴한다.

In [17]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)

Using label list ['0', '1']
*** Example ***
guid: 1680
features: InputFeatures(input_ids=[101, 1996, 7235, 9819, 2097, 2552, 2004, 20478, 21334, 2015, 1010, 6575, 2005, 3350, 1997, 2627, 2300, 1012, 102, 1996, 9819, 2552, 2004, 20478, 21334, 2015, 1010, 3048, 2006, 2416, 7787, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0,

- `tf_glue_convert_examples_to_features()` 함수가 최종적으로 모델에 전달될 `tf.data.Dataset` 인스턴스를 생성한다.  
- 위 코드는 그렇게 생성된 학습 단계 데이터셋 `train_dataset`이다.

In [18]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  1996,  7235,  9819,  2097,  2552,  2004, 20478, 21334,
        2015,  1010,  6575,  2005,  3350,  1997,  2627,  2300,  1012,
         102,  1996,  9819,  2552,  2004, 20478, 21334,  2015,  1010,
        3048,  2006,  2416,  7787,  1012,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

전체 데이터셋을 구성해보자.

In [19]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['0', '1']
*** Example ***
guid: 1680
features: InputFeatures(input_ids=[101, 1996, 7235, 9819, 2097, 2552, 2004, 20478, 21334, 2015, 1010, 6575, 2005, 3350, 1997, 2627, 2300, 1012, 102, 1996, 9819, 2552, 2004, 20478, 21334, 2015, 1010, 3048, 2006, 2416, 7787, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0,

In [20]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

Using label list ['0', '1']
*** Example ***
guid: 3155
features: InputFeatures(input_ids=[101, 1996, 2265, 1005, 1055, 8503, 5360, 2353, 1011, 4284, 16565, 2566, 3745, 2011, 1037, 10647, 1012, 102, 1996, 2194, 2056, 2023, 19209, 16565, 2011, 1037, 10647, 1037, 3745, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0

In [21]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

Using label list ['0', '1']
*** Example ***
guid: 163
features: InputFeatures(input_ids=[101, 6661, 1999, 8670, 2020, 2091, 1015, 1012, 1019, 3867, 2012, 16923, 7279, 3401, 2011, 16087, 2692, 13938, 2102, 1010, 2125, 1037, 2659, 1997, 17943, 2361, 1010, 1999, 1037, 3621, 6428, 3452, 2414, 3006, 1012, 102, 6661, 1999, 8670, 2020, 2091, 2093, 3867, 2012, 13913, 1011, 1015, 1013, 1018, 7279, 3401, 2011, 5641, 22394, 13938, 2102, 1010, 2125, 1037, 2659, 1997, 17943, 7279, 3401, 1010, 1999, 1037, 6428, 3006, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## **2.3 Train/Evaluation과 Test**

### **2.3.1 tf.keras.model을 활용한 학습**

우선 우리에게 익숙한 `model.fit()`을 이용한 모델 학습을 진행해보자.

In [22]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print("슝~")

슝~


In [23]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


학습을 진행해보자. (편의상 2 epoch만 진행 후 성능 체크)  
이 학습은 이미 잘 훈련된 BERT 모델을 가져다가 fine-tuning하는 작업임을 기억하자.

In [24]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=2, steps_per_epoch=115, 
                validation_data=validation_dataset_batch)

Epoch 1/2
115/115 [==============================] - 70s 452ms/step - loss: 0.6001 - acc: 0.6886 - val_loss: 0.5334 - val_acc: 0.7279
Epoch 2/2
115/115 [==============================] - 50s 436ms/step - loss: 0.4946 - acc: 0.7533 - val_loss: 0.3664 - val_acc: 0.8137


학습이 잘 진행되었다면 아래와 같이 테스트 결과를 만들어 확인해 볼 수 있다.

In [25]:
result = model.evaluate(test_dataset_batch)
print(result)

108/108 [==============================] - 16s 149ms/step - loss: 0.3671 - acc: 0.7803
[0.36714208126068115, 0.7802898287773132]


In [27]:
import os
output_dir = '/content/drive/MyDrive/AIFFEL/[GD-16]/transformers'
output_eval_file = os.path.join(output_dir, "eval_results.txt")

with open(output_eval_file, "w") as writer:
    for i, v in enumerate(result) :
        if i == 0 :
            writer.write("Loss = %f\t" %(v))
        if i == 1 :
            writer.write("Accuracy = %f\n" %(v))
print("완료=3")

#파일에 쓴 테스트 결과 확인
!cat ~/content/drive/MyDrive/AIFFEL/[GD-16]/transformers/eval_results.txt

완료=3
cat: '/root/content/drive/MyDrive/AIFFEL/[GD-16]/transformers/eval_results.txt': No such file or directory


### **2.3.2 TFTrainer를 활용한 학습**

이번에는 Huggingface의 `TFTrainer`를 활용해 학습을 진행해보자.  
이전 노드에서 살펴본 것처럼 `TFTrainer`를 활용하기 위해서는 `TFTrainingArguments`에 학습 관련 설정을 미리 지정해두어야 한다.

In [29]:
# TFTrainer을 활용하는 형태로 모델 재생성
from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    glue_compute_metrics,
    glue_convert_examples_to_features,
    glue_output_modes,
    glue_processors,
    glue_tasks_num_labels,
)

output_dir = '/content/drive/MyDrive/AIFFEL/[GD-16]/transformers'

training_args = TFTrainingArguments(
    output_dir=output_dir,            # output이 저장될 경로
    num_train_epochs=3,              # train 시킬 총 epochs
    per_device_train_batch_size=16,  # 각 device 당 batch size
    per_device_eval_batch_size=64,   # evaluation 시에 batch size
    warmup_steps=500,                # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,                 # weight decay
    logging_dir='./logs',                 # log가 저장될 경로
    do_train=True,
    do_eval=True,
    eval_steps=1000
)

max_seq_length=128
task_name = "mrpc"

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


아래에서 생성하게 될 `TFTrainer`의 인자로 넘겨주어야 할 것 중에 `compute_metrics` 메소드가 있다.  
이것은 task가 classification인지 regression인지에 따라 모델의 출력 상태가 달라지므로 task 별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정해 두는 것이다.

In [30]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return glue_compute_metrics(task_name, preds, p.label_ids)

output_mode = glue_output_modes[task_name]
output_mode

'classification'

`TFTrainer`를 활용할 때 데이터셋에서 잊지 않아야 할 것이 `tf.data.experimental.assert_cardinality()`를 데이터셋에 적용해 주는 것이다.  
이를 호출해 주지 않으면 `TFTrainer.train()`에서 assert fail이 발생한다.

In [31]:
train_dataset = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))
validation_dataset = validation_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation'].num_examples))
test_dataset = test_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test'].num_examples))

print("슝~")

슝~


이제 `TFTrainer`를 생성해서 본격적으로 학습을 시작해 보자.

In [32]:
trainer = TFTrainer(
    model=model,                           # 학습시킬 model
    args=training_args,                  # TFTrainingArguments을 통해 설정한 arguments
    train_dataset=train_dataset,    # training dataset
    eval_dataset=validation_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)

print("슝~")

슝~


/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:113: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  FutureWarning,


In [33]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

print("완료")

완료


학습이 끝나면 Evaluation을 진행하여 위 `model.fit()`으로 학습한 경우와 비교해 보자.

In [35]:
import numpy as np

# Evaluation
results = {}
if training_args.do_eval:
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results2.txt")

    with open(output_eval_file, "w") as writer:
        for key, value in result.items():
            writer.write(f"{key} = {value}\n")

        results.update(result)
        
#파일에 쓴 테스트 결과 확인
!cat ~/content/drive/MyDrive/AIFFEL/[GD-16]/transformers/eval_results2.txt

## ▼ 'eval_results2.txt'에 있는 결과
# eval_loss = 0.43034097126552034
# eval_acc = 0.8459821428571429
# eval_f1 = 0.8962406015037595
# eval_acc_and_f1 = 0.8711113721804512

/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:42: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled

cat: '/root/content/drive/MyDrive/AIFFEL/[GD-16]/transformers/eval_results2.txt': No such file or directory
